In [1]:
from huggingface_hub import InferenceClient

In [2]:
client = InferenceClient("meta-llama/Llama-3.2-11B-Vision-Instruct")

In [3]:
output = client.text_generation(
    "The capital of France is",
    max_new_tokens=100,
)

print(output)

 Paris, which is located in the northern part of the country. Paris is known for its beautiful architecture, art museums, fashion, and romantic atmosphere. The city is home to many famous landmarks such as the Eiffel Tower, Notre Dame Cathedral, and the Louvre Museum.
The official language of France is French, but many people in Paris also speak English, especially in the tourism industry. The currency used in France is the Euro.
Paris is a popular tourist destination, and it's easy to


In [ ]:
prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
The capital of France is<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
output = client.text_generation(
    prompt,
    max_new_tokens=100,
)

print(output)


Paris.


In [12]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of France is"},
    ],
    stream=False,
    max_tokens=1024,
)

print(output.choices[0].message.content)

Paris.
